<a href="https://colab.research.google.com/github/kyrillosishak/SentimentAnalysis/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! gdown 1aucr_u_8sKcfuDNhFbB6IzbpOsInHd9d
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Downloading...
From: https://drive.google.com/uc?id=1aucr_u_8sKcfuDNhFbB6IzbpOsInHd9d
To: /content/kaggle.json
100% 69.0/69.0 [00:00<00:00, 324kB/s]


In [2]:
!kaggle datasets download -d bittlingmayer/amazonreviews

100% 493M/493M [00:14<00:00, 43.4MB/s]
100% 493M/493M [00:14<00:00, 34.8MB/s]


In [3]:
!unzip amazonreviews.zip

Archive:  amazonreviews.zip
  inflating: test.ft.txt.bz2         
  inflating: train.ft.txt.bz2        


In [4]:
!bzip2 -dk train.ft.txt.bz2

In [6]:
!cd /content/

In [13]:
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader,TensorDataset
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Check if a GPU is available, and use it if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [6]:
# Check if a GPU is available, and use it if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
device

device(type='cuda', index=0)

In [2]:
# Specify the file path where the text is stored
file_path = '/content/train.ft.txt'

# Read the text from the file
with open(file_path, 'r') as file:
    text = file.read()

# Split the text into lines based on the '__label__' delimiter
lines = text.split('\n__label__')[1:]

# Initialize lists to store labels, subjects, and reviews
labels = []
subjects = []
reviews = []
Count = 0
# Split each line into label, subject, and review
for line in lines:
    if Count >= 719999:
      break
    label, review = line.split(' ', 1)
    labels.append(int(label) - 1)

    # Extract subject from review
    subject, review = review.split(': ', 1)
    subjects.append(subject)

    no_punctuation = re.sub(r'[\W]', r' ', review.lower())
    no_non_ascii = re.sub(r'[^a-z0-1\s]', r'', no_punctuation)
    reviews.append(no_non_ascii)
    Count+=1

# Create a DataFrame
df = pd.DataFrame({'Label': labels, 'Subject': subjects, 'Review': reviews})
del labels
del subjects
del reviews
del text
del lines

In [3]:
# Display the DataFrame
print(df.head())

   Label                                            Subject  \
0      1              The best soundtrack ever to anything.   
1      1                                           Amazing!   
2      1                               Excellent Soundtrack   
3      1  Remember, Pull Your Jaw Off The Floor After He...   
4      1                            an absolute masterpiece   

                                              Review  
0  i m reading a lot of reviews saying that this ...  
1  this soundtrack is my favorite music of all ti...  
2  i truly like this soundtrack and i enjoy video...  
3  if you ve played the game  you know how divine...  
4  i am quite sure any of you actually taking the...  


In [7]:
df = df.sample(frac=0.2, random_state=42)  # You can change the random_state for reproducibility


In [8]:
df.reset_index(drop=True, inplace=True)

In [9]:
df

,Label,Subject,Review
0,1,"An illuminating, touching and very funny read.",when i discovered that bradford dillman had wr...
1,0,Silly.............,aside from the two attractive leads there is l...
2,1,Gorgeous!,got these from my husband for christmas and th...
3,0,loved my old zyliss press but...,had a zyliss press for years that we loved t...
4,1,One Of My Most Favorite Movies!,i saw it on the amazon prime free instant play...
...,...,...,...
28795,1,More than thought provoking,my sister in law recommended this film to me ...
28796,0,Guess I did not read the label,clover honey with lavender flavor injected no...
28797,0,Only use if original pins are broken.,i ordered these to replace my worn pins on my ...
28798,1,Fibblestax,wonderful thought filled heart lifting story o...


In [10]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(df['Review'], df['Label'], train_size=0.75)
del df
# Initialize a CountVectorizer
cv = CountVectorizer(binary=True)
cv.fit(X_train)

# Transform the text data into feature vectors
X_train = cv.transform(X_train)
X_val = cv.transform(X_val)

In [11]:
# Create PyTorch tensors from your data
X_train = torch.tensor(X_train.toarray(), dtype=torch.float32).to(device)
X_val = torch.tensor(X_val.toarray(), dtype=torch.float32).to(device)
y_train = torch.tensor(y_train.values, dtype=torch.int64).to(device)
y_val = torch.tensor(y_val.values, dtype=torch.int64).to(device)

In [14]:
# Create PyTorch data loaders
batch_size = 64
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


In [15]:
# Define a logistic regression model in PyTorch and move it to GPU
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        out = self.linear(x)
        return out

In [17]:
# Define your logistic regression model
input_size = X_train.shape[1]
num_classes = 2  # Two classes: positive and negative

model = LogisticRegressionModel(input_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 40
for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        _, predicted = torch.max(val_outputs, 1)
        val_accuracy = accuracy_score(y_val.cpu().numpy(), predicted.cpu().numpy())
        print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Accuracy: {val_accuracy * 100:.2f}%')

# Testing the model
test_outputs = model(X_val)
_, predicted = torch.max(test_outputs, 1)
test_accuracy = accuracy_score(y_val.cpu().numpy(), predicted.cpu().numpy())
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Epoch [1/40], Validation Accuracy: 74.49%
Epoch [2/40], Validation Accuracy: 75.90%
Epoch [3/40], Validation Accuracy: 76.53%
Epoch [4/40], Validation Accuracy: 77.64%
Epoch [5/40], Validation Accuracy: 78.11%
Epoch [6/40], Validation Accuracy: 78.78%
Epoch [7/40], Validation Accuracy: 78.89%
Epoch [8/40], Validation Accuracy: 79.53%
Epoch [9/40], Validation Accuracy: 79.85%
Epoch [10/40], Validation Accuracy: 80.15%
Epoch [11/40], Validation Accuracy: 80.31%
Epoch [12/40], Validation Accuracy: 80.56%
Epoch [13/40], Validation Accuracy: 80.94%
Epoch [14/40], Validation Accuracy: 81.00%
Epoch [15/40], Validation Accuracy: 81.24%
Epoch [16/40], Validation Accuracy: 81.50%
Epoch [17/40], Validation Accuracy: 81.50%
Epoch [18/40], Validation Accuracy: 81.85%
Epoch [19/40], Validation Accuracy: 81.85%
Epoch [20/40], Validation Accuracy: 81.99%
Epoch [21/40], Validation Accuracy: 81.90%
Epoch [22/40], Validation Accuracy: 82.10%
Epoch [23/40], Validation Accuracy: 82.28%
Epoch [24/40], Valid





---



---



---



---



---



---



---


---



---


---



In [ ]:
import bz2
from collections import Counter
import re
import nltk
import numpy as np
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
train_file = bz2.BZ2File('train.ft.txt.bz2')
test_file = bz2.BZ2File('test.ft.txt.bz2')

In [ ]:
train_file = train_file.readlines()
test_file = test_file.readlines()

In [ ]:
pct = 0.3
num_train = int(len(train_file)*pct) #max 3600000
num_test = int(len(test_file)*pct) #max 400000
train_file = [x.decode('utf-8') for x in train_file[:num_train]]
test_file = [x.decode('utf-8') for x in test_file[:num_train]]
train_file[1:5]

["__label__2 The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny.\n",
 '__label__2 Amazing!: This soundtrack is my favorite music of all time, hands down. The intense sadness of "Prisoners of Fate" (which means all the more if you\'ve played the game) and the hope in "A Distant Promise" and "Girl who Stole the Star" have been an important inspiration to me personally throughout my teen years. The higher energy tracks like "Chrono Cross ~ Time\'s Scar~", "Time of the Dreamwatch", and "Chronomantique" (indefinably remeniscent of Chrono Tri

In [ ]:
# Extracting labels from sentences
train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_file]
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_file]

test_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test_file]
test_sentences = [x.split(' ', 1)[1][:-1].lower() for x in test_file]

In [ ]:
for i in range(len(train_sentences)):
    train_sentences[i] = re.sub('\d','0',train_sentences[i])

for i in range(len(test_sentences)):
    test_sentences[i] = re.sub('\d','0',test_sentences[i])

In [ ]:
for i in range(len(train_sentences)):
    if 'www.' in train_sentences[i] or 'http:' in train_sentences[i] or 'https:' in train_sentences[i] or '.com' in train_sentences[i]:
        train_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", train_sentences[i])

for i in range(len(test_sentences)):
    if 'www.' in test_sentences[i] or 'http:' in test_sentences[i] or 'https:' in test_sentences[i] or '.com' in test_sentences[i]:
        test_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", test_sentences[i])

In [ ]:
words = Counter()  # Dictionary that will map a word to the number of times it appeared in all the training sentences
for i, sentence in enumerate(train_sentences):
    # The sentences will be stored as a list of words/tokens
    train_sentences[i] = []
    for word in nltk.word_tokenize(sentence):  # Tokenizing the words
        words.update([word.lower()])  # Converting all the words to lowercase
        train_sentences[i].append(word)
    if i%20000 == 0:
        print(str((i*100)/num_train) + "% done")
print("100% done")


0.0% done
1.8518518518518519% done
3.7037037037037037% done
5.555555555555555% done
7.407407407407407% done
9.25925925925926% done
11.11111111111111% done
12.962962962962964% done
14.814814814814815% done
16.666666666666668% done
18.51851851851852% done
20.37037037037037% done
22.22222222222222% done
24.074074074074073% done
25.925925925925927% done
27.77777777777778% done
29.62962962962963% done
31.48148148148148% done
33.333333333333336% done
35.18518518518518% done
37.03703703703704% done
38.888888888888886% done
40.74074074074074% done
42.592592592592595% done
44.44444444444444% done
46.2962962962963% done
48.148148148148145% done
50.0% done
51.851851851851855% done
53.7037037037037% done
55.55555555555556% done
57.407407407407405% done
59.25925925925926% done
61.111111111111114% done
62.96296296296296% done
64.81481481481481% done
66.66666666666667% done
68.51851851851852% done
70.37037037037037% done
72.22222222222223% done
74.07407407407408% done
75.92592592592592% done
77.77777

In [ ]:
# Removing the words that only appear once
words = {k:v for k,v in words.items() if v>1}
# Sorting the words according to the number of appearances, with the most common word being first
words = sorted(words, key=words.get, reverse=True)
# Adding padding and unknown to our vocabulary so that they will be assigned an index
words = ['_PAD','_UNK'] + words
# Dictionaries to store the word to index mappings and vice versa
word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}
#convert the words in the sentences to their corresponding indexes

for i, sentence in enumerate(train_sentences):
    # Looking up the mapping dictionary and assigning the index to the respective words
    train_sentences[i] = [word2idx[word] if word in word2idx else 0 for word in sentence]

for i, sentence in enumerate(test_sentences):
    # For test sentences, we have to tokenize the sentences as well
    test_sentences[i] = [word2idx[word.lower()] if word.lower() in word2idx else 0 for word in nltk.word_tokenize(sentence)]

In [ ]:
#padding the sentences with 0s and shortening the lengthy sentences so that the data can be trained in batches to speed things up

# Defining a function that either shortens sentences or pads sentences with 0 to a fixed length
def pad_input(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

seq_len = 200  # The length that the sentences will be padded/shortened to

train_sentences = pad_input(train_sentences, seq_len)
test_sentences = pad_input(test_sentences, seq_len)

# Converting our labels into numpy arrays
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
#Split into train/Val/test datasets

split_frac = 0.5 # 50% validation, 50% test
split_id = int(split_frac * len(test_sentences))
val_sentences, test_sentences = test_sentences[:split_id], test_sentences[split_id:]
val_labels, test_labels = test_labels[:split_id], test_labels[split_id:]

In [ ]:
#Create model with Pytorch

import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

train_data = TensorDataset(torch.from_numpy(train_sentences), torch.from_numpy(train_labels))
val_data = TensorDataset(torch.from_numpy(val_sentences), torch.from_numpy(val_labels))
test_data = TensorDataset(torch.from_numpy(test_sentences), torch.from_numpy(test_labels))

batch_size = 400

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [ ]:
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print('GPU available')
else:
    device = torch.device("cpu")
    print('Only CPU')

GPU available


In [ ]:
class SentimentNet(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentNet, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, hidden):
        batch_size = x.size(0)
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        out = self.dropout(lstm_out)
        out = self.fc(out)
        out = self.sigmoid(out)

        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [ ]:
vocab_size = len(word2idx) + 1
output_size = 1
embedding_dim = 400
hidden_dim = 512
n_layers = 2

model = SentimentNet(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
model.to(device)

lr=0.005
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
epochs = 2
counter = 0
print_every = 500
clip = 5
valid_loss_min = np.Inf

model.train()
for i in range(epochs):
    h = model.init_hidden(batch_size)

    for inputs, labels in train_loader:
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        if counter%print_every == 0:
            val_h = model.init_hidden(batch_size)
            val_losses = []
            model.eval()
            for inp, lab in val_loader:
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp, val_h)
                val_loss = criterion(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())

            model.train()
            print("Epoch: {}/{}...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './state_dict.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)

Epoch: 1/2... Step: 500... Loss: 0.223739... Val Loss: 0.206195
Validation loss decreased (inf --> 0.206195).  Saving model ...
Epoch: 1/2... Step: 1000... Loss: 0.212604... Val Loss: 0.175982
Validation loss decreased (0.206195 --> 0.175982).  Saving model ...
Epoch: 1/2... Step: 1500... Loss: 0.191558... Val Loss: 0.170314
Validation loss decreased (0.175982 --> 0.170314).  Saving model ...
Epoch: 1/2... Step: 2500... Loss: 0.171799... Val Loss: 0.174236
Epoch: 2/2... Step: 3000... Loss: 0.183870... Val Loss: 0.167538
Validation loss decreased (0.170314 --> 0.167538).  Saving model ...
Epoch: 2/2... Step: 3500... Loss: 0.156412... Val Loss: 0.170313
Epoch: 2/2... Step: 4000... Loss: 0.147273... Val Loss: 0.169157
Epoch: 2/2... Step: 4500... Loss: 0.145281... Val Loss: 0.179763
Epoch: 2/2... Step: 5000... Loss: 0.143815... Val Loss: 0.179764


In [ ]:
# Loading the best model
model.load_state_dict(torch.load('./state_dict.pt'))

test_losses = []
num_correct = 0
h = model.init_hidden(batch_size)

model.eval()
for inputs, labels in test_loader:
    h = tuple([each.data for each in h])
    inputs, labels = inputs.to(device), labels.to(device)
    output, h = model(inputs, h)
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    pred = torch.round(output.squeeze())  # Rounds the output to 0/1
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

print("Test loss: {:.3f}".format(np.mean(test_losses)))
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}%".format(test_acc*100))

Test loss: 0.163
Test accuracy: 93.746%
